In [ ]:
!pip install matplotlib mpmath numpy pandas sympy bokeh

In [ ]:
from google.colab import drive

drive.mount("/content/gdrive", force_remount=True)


Mounted at /content/gdrive


## For interactive Bokeh plot this code has to be inside the cell of the Bokeh code.

In [ ]:
from bokeh.io import output_notebook
output_notebook()

-------------------------------------------------------------------------------------------------------------------




---



**AFTER THIS POINT YOU CAN START WORKING**

---








Sure, here is a code you can use for the Bokeh server:

In [ ]:
# Code for app.py
# Import necessary libraries
import pandas as pd
from bokeh.plotting import figure, curdoc
from bokeh.models import ColumnDataSource, CheckboxButtonGroup
from bokeh.layouts import column

# Load the dataset
df = pd.read_csv("data_science_salaries_2023.csv")
df['work_year'] = pd.to_datetime(df['work_year'], format='%Y')

# Create a ColumnDataSource and populate it with the entire DataFrame
avg_salary_by_country = df.groupby('work_year')['salary_in_usd'].mean().reset_index()
source = ColumnDataSource(data=dict(
    work_year=df['work_year'],
    avg_salary=df['salary_in_usd']
))

# Create the figure
p = figure(x_range=['2020','2021','2022','2023'], min_height=280, min_width=800, title="Average Salary in USD by Experience Level")

# Customize the plot style
p.yaxis.axis_label = "Average Salary (USD)"
p.xaxis.axis_label = "Year"

experience_levels = list(df['experience_level'].unique())

df_exp = [
    df[df['experience_level'] == exp_level].groupby('work_year')['salary_in_usd'].mean().reset_index()
    for exp_level in experience_levels
    ]



experience_data = [
    ColumnDataSource(data=dict(
        work_year=[str(year.year) for year in dfe['work_year']],
        avg_salary=dfe['salary_in_usd']
    )) for dfe in df_exp
]


# Plot each line for each experience level
lines = {}
for exp, source in enumerate(experience_data):
    lines[exp] = p.line('work_year', 'avg_salary',
                            source=source, line_width=2)

# Create the checkbox button group
checkbox_group = CheckboxButtonGroup(
    labels=experience_levels,
    active=list(range(len(experience_levels))))

# Define a callback function to update line visibility based on
# checkbox selection
def update_visibility(attr, old, new):
    """
    Toggle the visibility of each line that is active or inactive in the experience level checkbox button group.
    """

    for i in checkbox_group.active:
        lines[i].visible = True
    for i in range(len(experience_levels)):
        if i not in checkbox_group.active:
            lines[i].visible = False

checkbox_group.on_change('active', update_visibility)

# Call the update_plot function initially to populate the plot
update_visibility(None, None, None)

# Arrange the plot and the Select widget in a layout
layout = column(checkbox_group, p)

# Add the layout to the current document (Bokeh server)
curdoc().add_root(layout)

### **You can save this code to a file named `app.py`, and then run it from the command-line like this:**

### **`bokeh server --show app.py`**